In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import os
import matplotlib.pyplot as plt
import nltk
from collections import OrderedDict

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from collections import OrderedDict

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to /home/riya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/riya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/riya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from keybert import KeyBERT

In [3]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api
import matplotlib.pyplot as plt
import spacy
import string
from sklearn.metrics.pairwise import cosine_similarity

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
#list(gensim.downloader.info()['models'].keys())
#wv = api.load('glove-wiki-gigaword-300')
#wv.save('glove_wiki_vectors.kv')

In [5]:
revalive_path_vec = "../../glove_wiki_vectors.kv"
relative_path_sdg = "../../sdg_data.csv"
relative_path_attr = "../Dictionary/Attributes_2016-17.csv"
relative_string_map = "./Data_Target/String_map.csv"
relative_string_map_unroll = "./Data_Target/String_map_unroll.csv"
relative_semantic = "./Data_Target/Semantic_map.csv"
relative_semantic_unroll = "./Data_Target/Semantic_map_unroll.csv"
relative_sem_threshold = "./Data_Target/Semantic_threshold_map.csv"
relative_sem_threshold_unroll = "./Data_Target/Semantic_threshold_map_unroll.csv"

# Get the absolute path by joining the current directory with the relative path
absolute_path_vec = os.path.normpath(os.path.join(os.getcwd(), revalive_path_vec))
absolute_path_sdg = os.path.normpath(os.path.join(os.getcwd(), relative_path_sdg))
absolute_path_attr = os.path.normpath(os.path.join(os.getcwd(), relative_path_attr))
absolute_path_string = os.path.normpath(os.path.join(os.getcwd(), relative_string_map))
absolute_path_string_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_string_map_unroll))
absolute_path_semantic = os.path.normpath(os.path.join(os.getcwd(), relative_semantic))
absolute_path_semantic_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_semantic_unroll))
absolute_path_sem_threshold =os.path.normpath( os.path.join(os.getcwd(), relative_sem_threshold))
absolute_path_sem_threshold_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_sem_threshold_unroll))

In [6]:
wv = KeyedVectors.load(absolute_path_vec)

In [7]:
kw_model = KeyBERT(model=wv)

In [8]:
def replace_words(df, column_list, replace_dict):
    for col in column_list:
        # Apply the replace function to the specified column using the replace_dict
        df[col] = df[col].replace(replace_dict, regex=True)
    return df

# Define the replacement dictionary
replace_dict = {
    'B.p.l.': 'below poverty line',
    'A.p.l.': 'above poverty line',
    'Bpl': 'below poverty line',
}

In [9]:
IGNORE_WORDS = ['proportion', 'whose', 'number', 'per', 'within', 'no', 'under', 'volume', 'use', 'type', 'percentage', 'total', 'data', 'year', 'due', 'rate', 'hects', 'hect', 'noof', 'years']

In [10]:
# Creating our tokenizer function
def cleaning(sentence):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    wn = nltk.WordNetLemmatizer()
    ignore_word = []
    cleaned = []
    for i in range(len(sentence)):
        letters_1 = re.sub('[^a-zA-Z.]', ' ', sentence[i])
        letters = re.sub(r'\.', '', letters_1)
        letters = letters.lower().split() 
        letters = [wn.lemmatize(w) for w in letters if w not in stopwords ]
        IGNORE_WORDS_LEM = [wn.lemmatize(w).lower() for w in IGNORE_WORDS]
        letters = [w for w in letters if w not in IGNORE_WORDS_LEM]
        #letters_list = list(set(letters))
        letters_list = list(OrderedDict.fromkeys(letters))   
        cleaned.append(letters_list)

    return cleaned

In [11]:
# Creating our tokenizer function
def concat_cleaning(sentence):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    wn = nltk.WordNetLemmatizer()

    cleaned = []
    for i in range(len(sentence)):
        letters_1 = re.sub('[^a-zA-Z.]', ' ', sentence[i])
        letters = re.sub(r'\.', '', letters_1)
        letters = letters.lower().split() 
        #letters = [wn.lemmatize(w) for w in letters if w not in stopwords ]
        #IGNORE_WORDS_LEM = [wn.lemmatize(w).lower() for w in IGNORE_WORDS]
        #letters = [w for w in letters if w not in IGNORE_WORDS_LEM]
        letters = ' '.join(letters)
        cleaned.append(letters)

    return cleaned

In [12]:
def process_keyword(data):
    result = []
    for sublist in data:
        inner_list = []
        for word, _ in sublist:
            # Split the word if it contains a space and reverse the order of words
            words = word.split()
            # Filter out numeric values
            words = [w for w in words if not w.isdigit()]
            if words:  # Exclude empty lists
                inner_list.append(words)
        inner_list.reverse()
        result.append(inner_list)
    return result

In [13]:
def add_elements(row):
    if not row['keywords']:  # Check if col2 is empty
        row['keywords'].extend([[elem] for elem in row['new_indicators']])  # Add elements from col1 to col2 as lists
    return row

In [14]:
def concatenate_table_name(df):
    for index, row in df.iterrows():
        if ',' not in row['Description']:
            df.at[index, 'Description'] += ', ' + row['Table_name']
        elif row['Description'].count(',') == 1:
            parts = row['Description'].split(',')
            if len(parts) == 2:
                part1, part2 = parts
                if re.match(r'^\w+$', part1.strip()) or re.match(r'^\w+$', part2.strip()):
                    df.at[index, 'Description'] += ', ' + row['Table_name']
    return df


In [15]:
def concatenate_all(df):
    for index, row in df.iterrows():
        df.at[index, 'Description'] += ', ' + row['Table_name'] + ', ' + row['Chapter_name'] 
    return df

In [16]:
missing_words = []
miss_added = []
def sent_vec(row):
    sent = row['new_description']
    cap_tab = row['Chapter_name'] + ", " + row['Table_name']
    vector_size = wv.vector_size
    vectors = []
    miss_w = []
    for w in sent:
        if w in wv:
            wv_res = np.zeros(vector_size)
            wv_res = wv[w]
            vectors.append(wv_res)
        else:
            missing_words.append(("KAG", w))
            miss_w.append(w)
    if len(miss_w)>0:
        vectors.append(missin_word_vec(cap_tab))
        miss_added.append(miss_w)
    vec_arr = np.array(vectors)
    len_vec = len(vectors)
    return vec_arr

In [17]:
def missin_word_vec(sent):
    vector_size = wv.vector_size
    sent_list = cleaning([sent])
    wv_res = np.zeros(vector_size)
    ctr = 0
    for w in sent_list[0]:
        if w in wv:
            wv_res += wv[w]
            ctr += 1
    if ctr>0:
        wv_res = wv_res/ctr
    else:
        wv_res = np.random.rand(vector_size)
        print('yes')
    return wv_res

In [18]:
def sent_vec_avg(sent):
    vector_size = wv.vector_size
    missing_words = []
    vectors = []
    for w in sent:
        ctr = 0
        wv_res = np.zeros(vector_size)
        for word in  w:
            if word in wv:
                wv_res += wv[word]
                ctr += 1
            else:
                missing_words.append(word)
        if ctr>0:
            wv_res = wv_res/ctr
        vectors.append(wv_res)
    vec_arr = np.array(vectors)
    len_vec = len(vectors)
    return vec_arr

In [19]:
def count_common_words(row, df_other):
    set_1 = set(row)
    common_words = []
    for _, other_row in df_other.iterrows():
        set_2 = set(other_row['new_description'])
        common_words.append(len(set_1.intersection(set_2)))
    return common_words

In [20]:
def find_similarity_string(similarity_matrix, threshold, chapter_df, sdg_df):
    top_n_similarities = []
    top_n_attr = []
    top_n_chapter_id = []
    top_n_chapter_name = []
    top_n_table_id = []
    top_n_table_name = []
    top_n_count = []
    top_n_description = []
    
    for index, row in similarity_matrix.iterrows():
        # Filter indices based on threshold value
        top_n_indices = [i for i, value in enumerate(row) if value >= threshold]
        
        # Sort indices based on similarity values
        top_n_indices = sorted(top_n_indices, key=lambda i: row.iloc[i], reverse=True)
        
        # Get top n values, chapters, and chapter names
        values = [row.iloc[i] for i in top_n_indices]
        attr = [similarity_matrix.columns[i] for i in top_n_indices]
        attr_des = [chapter_df['Description'].loc[i] for i in top_n_indices]
        chapter_id = [chapter_df['Chapter_id'].loc[i] for i in top_n_indices]
        chapter_name = [chapter_df['Chapter_name'].loc[i] for i in top_n_indices]   
        table_id = [chapter_df['Table_id'].loc[i] for i in top_n_indices]
        table_name = [chapter_df['Table_name'].loc[i] for i in top_n_indices]   


        # Append to respective lists
        top_n_count.append(len(attr))
        top_n_similarities.append(values)
        top_n_attr.append(attr)
        top_n_description.append(attr_des)
        top_n_chapter_id.append(chapter_id)
        top_n_chapter_name.append(chapter_name)
        top_n_table_id.append(table_id)
        top_n_table_name.append(table_name)
    
    # Assign lists to the SDG DataFrame
    sdg_df['top_n_count'] = top_n_count
    sdg_df['top_n_attr'] = top_n_attr
    sdg_df['top_n_table_id'] = top_n_table_id
    sdg_df['top_n_table_name'] = top_n_table_name
    sdg_df['top_n_chapter_id'] = top_n_chapter_id
    sdg_df['top_n_chapter_name'] = top_n_chapter_name
    sdg_df['top_n_similarities'] = top_n_similarities
    sdg_df['top_n_description'] = top_n_description

    return sdg_df


In [21]:
def process_result_string(result_sdg, sdg_df):
    rows = []
    
    for _, row in result_sdg.iterrows():
        attr_id = row['top_n_attr']
        chapter_id = row['top_n_chapter_id']
        table_id = row['top_n_table_id']
        values = row['top_n_similarities']
        attr_desc = row['top_n_description']
        chapter_name = row['top_n_chapter_name']
        table_name = row['top_n_table_name']
        goal_no = row['Goal No.']
        target_id = row['Target_id']
        
        for attr, chap, tab, val, desc, chap_name, tab_name in zip(
            attr_id, chapter_id, table_id, values, attr_desc, chapter_name, table_name
        ):
            rows.append({
                'Attr_id': attr,
                'Chapter_id': chap,
                'Chapter_name': chap_name,
                'Table_id': tab,
                'Table_name': tab_name,
                'Description': desc,
                'top_n_goal_id': goal_no,
                'top_n_target_id': target_id,
                'top_n_similarities': val
            })

    all_goals = pd.DataFrame(rows)
    
    goal_name = dict(zip(sdg_df['Goal No.'], sdg_df['Goal']))
    target_name = dict(zip(sdg_df['Target_id'], sdg_df['Targets']))
    
    all_goals['Goal'] = all_goals['top_n_goal_id'].map(goal_name.get)
    all_goals['Targets'] = all_goals['top_n_target_id'].map(target_name.get)
    all_goals = all_goals.loc[:,['Attr_id','Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description', 'Goal', 'Targets', 'top_n_goal_id', 'top_n_target_id', 'top_n_similarities']]
    return all_goals


In [22]:
def find_similarity_semantic(sdg_df, att_df, threshold):
    ind_list = sdg_df['vec'].tolist()
    att_list = att_df['vec'].tolist()
    min_similarity_values = []
    top_indices_values = []
    for ind in range(len(ind_list)):
        top_indices_list = []
        top_sim_list = []
        top_min_list = []
        for i in ind_list[ind]:
            max_list = []
            for att in range(len(att_list)):
                similarity_matrix = cosine_similarity([i], att_list[att])
                max_similarity = np.max(similarity_matrix)
                max_list.append(max_similarity)
            # Get the indices of the top 10 max values
            #top_indices = [idx for idx, sim in enumerate(max_list) if sim < 1]
            top_indices = sorted(range(len(max_list)), key=lambda i: max_list[i], reverse=True)[:threshold]
            top_indices_list.append(top_indices)
            top_sim = [max_list[i] for i in top_indices]
            top_sim_list.append(top_sim)
            top_min_list.append(min(top_sim))
            
        # Merge the inner lists of l1 and l2
        merged_index = [item for sublist in top_indices_list for item in sublist]
        merged_sim = [item for sublist in top_sim_list for item in sublist]
        
        # Sort l1 based on the values in l2
        sorted_indices = sorted(range(len(merged_sim)), key=lambda i: merged_sim[i], reverse=True)
        sorted_index = [merged_index[i] for i in sorted_indices]
        final_index_list = list(OrderedDict.fromkeys(sorted_index))
            
        #top_indices_set = list(OrderedDict.fromkeys(top_indices_list))
        top_indices_values.append(final_index_list)
        min_similarity_values.append(top_min_list)
    return min_similarity_values, top_indices_values


In [23]:
#indicator words to attribute complete name
def find_similarity_semantic_threshold(sdg_df, att_df, diff):
    ind_list = sdg_df['vec'].tolist()
    att_list = att_df['vec'].tolist()
    min_list = sdg_df['min_values'].tolist()
    top_indices_values = []
    for ind in range(len(ind_list)):
        top_indices_list = []
        top_sim_list = []
        min_values = min_list[ind]
        for index, i in enumerate(ind_list[ind]):
            max_list = []
            threshold = min_values[index] - diff
            if(threshold <= 0):
                continue
            for att in range(len(att_list)):
                similarity_matrix = cosine_similarity([i], att_list[att])
                max_value = np.max(similarity_matrix)
                max_list.append((max_value, att))
            
            # Get the indices of values strictly greater than threshold
            top_indices = [idx for val, idx in max_list if val > threshold]
            # Sort top_indices based on the corresponding values in max_list
            top_indices.sort(key=lambda x: next((val for val, idx in max_list if idx == x), 0), reverse=True)
            top_indices_list.append(top_indices)
            top_sim = [max_list[i][0] for i in top_indices]
            top_sim_list.append(top_sim)
        # Merge the inner lists of l1 and l2
        merged_index = [item for sublist in top_indices_list for item in sublist]
        merged_sim = [item for sublist in top_sim_list for item in sublist]
        
        # Sort l1 based on the values in l2
        sorted_indices = sorted(range(len(merged_sim)), key=lambda i: merged_sim[i], reverse=True)
        sorted_index = [merged_index[i] for i in sorted_indices]
        final_index_list = list(OrderedDict.fromkeys(sorted_index))
        
        #top_indices_set = list(OrderedDict.fromkeys(top_indices_list))
        top_indices_values.append(final_index_list)
        
    return top_indices_values

In [24]:
def finding_attr_details(sdg_df, att_df):
    result_list = sdg_df['top_n_index'].tolist()
    top_n_count = []
    top_n_attr = []
    top_n_chapter_id = []
    top_n_table_id = []
    top_n_description = []
    top_n_chapter_name = []
    top_n_table_name = []
    for ll in result_list:
        top_n_count.append(len(ll))
        top_n_attr.append([att_df.loc[idx, 'Attr_id'] for idx in ll])
        top_n_chapter_id.append([att_df.loc[idx, 'Chapter_id'] for idx in ll])
        top_n_table_id.append([att_df.loc[idx, 'Table_id'] for idx in ll])
        top_n_description.append([att_df.loc[idx, 'Description'] for idx in ll])
        top_n_chapter_name.append([att_df.loc[idx, 'Chapter_name'] for idx in ll])
        top_n_table_name.append([att_df.loc[idx, 'Table_name'] for idx in ll])
    sdg_df['top_n_count'] = top_n_count
    sdg_df['top_n_attr'] = top_n_attr
    sdg_df['top_n_chapter_id'] = top_n_chapter_id
    sdg_df['top_n_table_id'] = top_n_table_id
    sdg_df['top_n_description'] = top_n_description
    sdg_df['top_n_chapter_name'] = top_n_chapter_name
    sdg_df['top_n_table_name'] = top_n_table_name
    return sdg_df

In [25]:
def process_result_semantic(result_sdg, sdg_df):
    rows = []
    
    for _, row in result_sdg.iterrows():
        attr_id = row['top_n_attr']
        chapter_id = row['top_n_chapter_id']
        table_id = row['top_n_table_id']
        attr_desc = row['top_n_description']
        chapter_name = row['top_n_chapter_name']
        table_name = row['top_n_table_name']
        goal_no = row['Goal No.']
        target_id = row['Target_id']
        
        for attr, chap, tab, desc, chap_name, tab_name in zip(
            attr_id, chapter_id, table_id, attr_desc, chapter_name, table_name
        ):
            rows.append({
                'Attr_id': attr,
                'Chapter_id': chap,
                'Chapter_name': chap_name,
                'Table_id': tab,
                'Table_name': tab_name,
                'Description': desc,
                'top_n_goal_id': goal_no,
                'top_n_target_id': target_id
            })

    all_goals = pd.DataFrame(rows)
    
    goal_name = dict(zip(sdg_df['Goal No.'], sdg_df['Goal']))
    target_name = dict(zip(sdg_df['Target_id'], sdg_df['Targets']))
    
    all_goals['Goal'] = all_goals['top_n_goal_id'].map(goal_name.get)
    all_goals['Targets'] = all_goals['top_n_target_id'].map(target_name.get)
    all_goals = all_goals.loc[:,['Attr_id','Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description', 'Goal', 'Targets', 'top_n_goal_id', 'top_n_target_id']]
    return all_goals


In [26]:
sdg = pd.read_csv(absolute_path_sdg, sep=",", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str})

In [27]:
sdg_complete = sdg.copy()

In [28]:
sdg

,Goal No.,Goal,Nodal Department,Targets,Other Related Major Departments,Tentative Indicators,Target_id,Indicator_id
0,1,End poverty in all its forms everywhere,Rural Development,"By 2030, eradicate extreme poverty for all peo...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population below the interna...,1.1,1.1.1
1,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living below the ...,1.2,1.2.1
2,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...","Proportion of men, women and children of all a...",1.2,1.2.2
3,1,End poverty in all its forms everywhere,Rural Development,Implement nationally appropriate social protec...,"Urban Development, Agriculture, Horticulture, ...",Percentage of the population covered by social...,1.3,1.3.1
4,1,End poverty in all its forms everywhere,Rural Development,"By 2030, ensure that all men and women, in par...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living in househo...,1.4,1.4.1
...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,Finance,"Promote the development, transfer, disseminati...",Environment and Scientific Technology.,Total amount of approved funding to promote th...,16.4,16.4.1
175,16,Strengthen the means of implementation and rev...,Finance,Fully operationalize the technology bank and s...,"Environment and Scientific Technology, Inform...",Proportion of individuals using the Internet.,16.5,16.5.1
176,17,"Data, monitoring and accountability",Finance,"By 2020, enhance capacity-building support to ...","Planning, Finance, Economic and Statistics.",Proportion of sustainable development indicato...,17.1,17.1.1
177,17,"Data, monitoring and accountability",Finance,"By 2030, build on existing initiatives to deve...","Planning, Finance, Economic and Statistics.",Dollar value of all resources made available t...,17.2,17.2.1


In [29]:
sdg.columns

Index(['Goal No.', 'Goal', 'Nodal Department', 'Targets',
       'Other Related Major Departments', 'Tentative Indicators', 'Target_id',
       'Indicator_id'],
      dtype='object')

In [30]:
sdg.drop(['Nodal Department','Other Related Major Departments', 'Tentative Indicators', 'Indicator_id'],axis=1,inplace=True)

In [31]:
# Remove duplicate rows
sdg = sdg.drop_duplicates()

In [32]:
sdg.reset_index(inplace = True, drop = True)

In [33]:
sdg

,Goal No.,Goal,Targets,Target_id
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5
...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, ...",16.3
127,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",16.4
128,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,16.5
129,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",17.1


In [34]:
sdg_new = sdg.copy()

In [35]:
texto_data=sdg_new['Targets'].tolist()
process_text=cleaning(texto_data)

cleaned_sent = pd.DataFrame({'new_targets': process_text})
sdg_new['new_targets']=cleaned_sent

In [36]:
texto_data0=sdg_new['Targets'].tolist()
process_text0=concat_cleaning(texto_data0)

cleaned_sent0 = pd.DataFrame({'new_targets_str': process_text0})
sdg_new['new_targets_str']=cleaned_sent0

In [37]:
sdg_new

,Goal No.,Goal,Targets,Target_id,new_targets,new_targets_str
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1,"[eradicate, extreme, poverty, people, everywhe...",by eradicate extreme poverty for all people ev...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2,"[reduce, least, half, men, woman, child, age, ...",by reduce at least by half the proportion of m...
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3,"[implement, nationally, appropriate, social, p...",implement nationally appropriate social protec...
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4,"[ensure, men, woman, particular, poor, vulnera...",by ensure that all men and women in particular...
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5,"[build, resilience, poor, vulnerable, situatio...",by build the resilience of the poor and those ...
...,...,...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, ...",16.3,"[enhance, cooperation, access, science, techno...",enhance cooperation on and access to science t...
127,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",16.4,"[promote, development, transfer, dissemination...",promote the development transfer dissemination...
128,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,16.5,"[fully, operationalize, technology, bank, scie...",fully operationalize the technology bank and s...
129,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",17.1,"[enhance, capacity, building, support, increas...",by enhance capacity building support to increa...


In [38]:
data = pd.read_csv(absolute_path_attr, sep=";", dtype={'Attr_id': str, 'Chapter_id': str, 'Table_id':str})

In [39]:
# Replace words in the DataFrame using the dictionary
data = replace_words(data, ['Chapter_name', 'Table_name', 'Description'], replace_dict)

In [40]:
data_new = concatenate_all(data)
data_new

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description
0,3.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Nada Offices, Nada Offices Village Accountant ..."
1,4.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Va Circles, Nada Offices Village Accountant Ci..."
2,5.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Hoblies, Nada Offices Village Accountant Circl..."
3,6.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Grama Panchayaths, Nada Offices Village Accoun..."
4,7.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"No.of Taluks, Nada Offices Village Accountant ..."
...,...,...,...,...,...,...
1162,1169.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Total,Number,Total Agrl. Land Holder Total, Ag..."
1163,1170.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Male,Area,Total Agrl. Land Holder Total, Agric..."
1164,1171.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Female,Area,Total Agrl. Land Holder Total, Agr..."
1165,1172.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Institutions,Area,Total Agrl. Land Holder Tota..."


In [41]:
# Create a new DataFrame with selected columns
new_df = data_new.copy()
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description
0,3.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Nada Offices, Nada Offices Village Accountant ..."
1,4.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Va Circles, Nada Offices Village Accountant Ci..."
2,5.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Hoblies, Nada Offices Village Accountant Circl..."
3,6.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Grama Panchayaths, Nada Offices Village Accoun..."
4,7.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"No.of Taluks, Nada Offices Village Accountant ..."
...,...,...,...,...,...,...
1162,1169.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Total,Number,Total Agrl. Land Holder Total, Ag..."
1163,1170.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Male,Area,Total Agrl. Land Holder Total, Agric..."
1164,1171.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Female,Area,Total Agrl. Land Holder Total, Agr..."
1165,1172.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Institutions,Area,Total Agrl. Land Holder Tota..."


In [42]:
texto_data1=new_df['Description'].tolist()
process_text1=cleaning(texto_data1)

cleaned_sent1 = pd.DataFrame({'new_description': process_text1})
new_df['new_description']=cleaned_sent1

In [43]:
texto_data11=new_df['Description'].tolist()
process_text11=concat_cleaning(texto_data11)

cleaned_sent11 = pd.DataFrame({'new_description_str': process_text11})
new_df['new_description_str']=cleaned_sent11

In [44]:
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,new_description,new_description_str
0,3.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Nada Offices, Nada Offices Village Accountant ...","[nada, office, village, accountant, circle, ho...",nada offices nada offices village accountant c...
1,4.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Va Circles, Nada Offices Village Accountant Ci...","[va, circle, nada, office, village, accountant...",va circles nada offices village accountant cir...
2,5.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Hoblies, Nada Offices Village Accountant Circl...","[hoblies, nada, office, village, accountant, c...",hoblies nada offices village accountant circle...
3,6.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Grama Panchayaths, Nada Offices Village Accoun...","[grama, panchayaths, nada, office, village, ac...",grama panchayaths nada offices village account...
4,7.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"No.of Taluks, Nada Offices Village Accountant ...","[taluks, nada, office, village, accountant, ci...",noof taluks nada offices village accountant ci...
...,...,...,...,...,...,...,...,...
1162,1169.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Total,Number,Total Agrl. Land Holder Total, Ag...","[agrl, land, holder, agricultural, holding, ar...",total number total agrl land holder total agri...
1163,1170.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Male,Area,Total Agrl. Land Holder Total, Agric...","[male, area, agrl, land, holder, agricultural,...",male area total agrl land holder total agricul...
1164,1171.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Female,Area,Total Agrl. Land Holder Total, Agr...","[female, area, agrl, land, holder, agricultura...",female area total agrl land holder total agric...
1165,1172.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Institutions,Area,Total Agrl. Land Holder Tota...","[institution, area, agrl, land, holder, agricu...",institutions area total agrl land holder total...


String Matching

In [45]:
common_word_matrix = sdg_new['new_targets'].apply(count_common_words, args=(new_df,))
common_word_matrix

0      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                             ...                        
126    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
127    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
128    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
129    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
130    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: new_targets, Length: 131, dtype: object

In [46]:
attr_id = new_df['Attr_id'].tolist()
ind_id = sdg_new['Target_id'].tolist()

In [47]:
# Determine the maximum number of words in any list
max_words = max(common_word_matrix, key=lambda x: len(x))

# Create DataFrame with columns Word_1, Word_2, ..., Word_n
common_word_df = pd.DataFrame(common_word_matrix.tolist(), columns=attr_id, index=ind_id)

In [48]:
common_word_df

,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,...,1164.0,1165.0,1166.0,1167.0,1168.0,1169.0,1170.0,1171.0,1172.0,1173.0
1.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1.5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16.3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16.4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16.5,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
17.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
# Example usage:
threshold_value = 1  # Define the threshold value

# Call the function to find chapter_ids with similarity greater than the threshold
sdg_string_map = find_similarity_string(common_word_df, threshold_value, new_df, sdg_new.copy())


In [50]:
sdg_string_map

,Goal No.,Goal,Targets,Target_id,new_targets,new_targets_str,top_n_count,top_n_attr,top_n_table_id,top_n_table_name,top_n_chapter_id,top_n_chapter_name,top_n_similarities,top_n_description
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1,"[eradicate, extreme, poverty, people, everywhe...",by eradicate extreme poverty for all people ev...,30,"[28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35....","[1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.7, 1.7, 1.7, ...",[Ration Shops And below poverty line Ration Ca...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General Information, General Information, Gen...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Urban,Ration shops, Ration Shops And below po..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2,"[reduce, least, half, men, woman, child, age, ...",by reduce at least by half the proportion of m...,94,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[10.8, 10.8, 10.8, 10.8, 10.8, 10.8, 13.1, 13....",[No .of Children in the age group 6 to 14 year...,"[10, 10, 10, 10, 10, 10, 13, 13, 13, 13, 13, 1...","[Education, Education, Education, Education, E...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[Boys,No.of Children Enrolment age group betwe..."
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3,"[implement, nationally, appropriate, social, p...",implement nationally appropriate social protec...,156,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[11.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11....",[Allopathy Hospitals Indian System No.of Medic...,"[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 4, 10...","[Health & Family welfare Services, Health & Fa...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...","[Nos.,Allopathy Hospitals, Allopathy Hospitals..."
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4,"[ensure, men, woman, particular, poor, vulnera...",by ensure that all men and women in particular...,336,"[667.0, 668.0, 669.0, 670.0, 670.1, 671.0, 672...","[11.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11....",[Allopathy Hospitals Indian System No.of Medic...,"[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1...","[Health & Family welfare Services, Health & Fa...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[Nos.,Allopathy Hospitals, Allopathy Hospitals..."
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5,"[build, resilience, poor, vulnerable, situatio...",by build the resilience of the poor and those ...,12,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17....","[Registration of Birth and Death, Registration...","[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]","[Miscellaneous, Miscellaneous, Miscellaneous, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[Birth,Registered events Year 2016, Registrati..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, ...",16.3,"[enhance, cooperation, access, science, techno...",enhance cooperation on and access to science t...,18,"[206.0, 523.0, 524.0, 525.0, 526.0, 527.0, 528...","[4.3, 9.3, 9.3, 9.3, 9.3, 9.3, 9.3, 11.3, 11.3...","[Area under principal crops Year 2015-16, Rura...","[4, 9, 9, 9, 9, 9, 9, 11, 11, 11, 11, 11, 11, ...","[Agriculture, Horticulture and Sericulture, TR...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Total Pulses including Tur Bengalgram,Pulses ..."
127,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",16.4,"[promote, development, transfer, dissemination...",promote the development transfer dissemination...,56,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[13.1, 13.1, 13.1, 13.1, 13.1, 13.2, 13.2, 13....",[Stree Shakti Self Hel

In [51]:
string_all_goals = process_result_string(sdg_string_map, sdg_new)

In [52]:
sdg_string_map.to_csv(absolute_path_string, sep=";", index=False)

In [53]:
string_all_goals.to_csv(absolute_path_string_unroll, sep=";", index=False)

Semantic Mapping

In [54]:
sdg_sem = sdg_new.copy()

In [55]:
word_count = sdg_sem['new_targets'].apply(lambda x: len(x))
word_count

0      10
1      13
2      14
3      27
4      12
       ..
126    19
127     8
128    14
129    26
130    16
Name: new_targets, Length: 131, dtype: int64

In [56]:
from collections import Counter
w_c = Counter(word_count)
w_c.most_common()

[(10, 10),
 (13, 10),
 (16, 10),
 (11, 10),
 (15, 10),
 (8, 8),
 (17, 7),
 (14, 6),
 (12, 6),
 (19, 6),
 (6, 6),
 (9, 6),
 (18, 5),
 (21, 4),
 (7, 4),
 (22, 3),
 (20, 3),
 (5, 3),
 (27, 2),
 (29, 2),
 (23, 2),
 (24, 2),
 (26, 2),
 (35, 1),
 (36, 1),
 (25, 1),
 (30, 1)]

In [57]:
w_c.most_common(10)

[(10, 10),
 (13, 10),
 (16, 10),
 (11, 10),
 (15, 10),
 (8, 8),
 (17, 7),
 (14, 6),
 (12, 6),
 (19, 6)]

In [58]:
np.mean(word_count)

14.65648854961832

In [59]:
median_indicator = np.median(word_count)
print(median_indicator)

14.0


In [60]:
from statistics import mode
mode_indicator = mode(word_count)
mode_indicator

10

In [61]:
stopwords_set = list(set(nltk.corpus.stopwords.words('english')))
stopwords_set.extend(IGNORE_WORDS)
stopwords_set = list(set(stopwords_set))
stopwords_set

['between',
 'total',
 'by',
 'couldn',
 'further',
 "hasn't",
 'mightn',
 'him',
 'there',
 'more',
 'shan',
 "wouldn't",
 'so',
 'they',
 'own',
 'out',
 'proportion',
 'hects',
 "weren't",
 'before',
 'on',
 'down',
 "wasn't",
 "doesn't",
 'use',
 'why',
 'for',
 'had',
 'being',
 'who',
 'how',
 'other',
 'data',
 'not',
 'then',
 "needn't",
 'y',
 'our',
 'over',
 'when',
 'themselves',
 'should',
 "it's",
 'haven',
 'were',
 'yourselves',
 'very',
 'isn',
 'doing',
 'while',
 'my',
 'too',
 'type',
 'or',
 'o',
 'below',
 'years',
 's',
 'because',
 'and',
 'same',
 'the',
 'having',
 'was',
 'in',
 'year',
 'herself',
 'with',
 'at',
 'hers',
 'doesn',
 'wouldn',
 'whose',
 've',
 'rate',
 'as',
 'don',
 'of',
 'about',
 'up',
 'after',
 "that'll",
 'mustn',
 'all',
 'no',
 'each',
 'd',
 'few',
 'if',
 'some',
 'ma',
 'but',
 'didn',
 'hasn',
 'myself',
 'have',
 'both',
 'his',
 "you'd",
 'them',
 'against',
 'these',
 'theirs',
 'from',
 'a',
 'himself',
 'nor',
 'needn',
 "d

In [62]:
#using 'new_indicators_str', takin top_n=6 as median = 8 and nr_candidates=10 as mean = 9.2
sdg_sem['keywords']=kw_model.extract_keywords(sdg_sem['new_targets_str'].tolist(), keyphrase_ngram_range=(1,2), stop_words = stopwords_set, use_maxsum=True, nr_candidates=10, top_n=6)

In [63]:
sdg_sem['keywords'] = process_keyword(sdg_sem['keywords'].tolist())
sdg_sem

,Goal No.,Goal,Targets,Target_id,new_targets,new_targets_str,keywords
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1,"[eradicate, extreme, poverty, people, everywhe...",by eradicate extreme poverty for all people ev...,"[[poverty, people], [measured, people], [peopl..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2,"[reduce, least, half, men, woman, child, age, ...",by reduce at least by half the proportion of m...,"[[reduce, least], [according, national], [acco..."
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3,"[implement, nationally, appropriate, social, p...",implement nationally appropriate social protec...,"[[appropriate, social], [protection, systems],..."
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4,"[ensure, men, woman, particular, poor, vulnera...",by ensure that all men and women in particular...,"[[well], [particular, poor], [financial, servi..."
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5,"[build, resilience, poor, vulnerable, situatio...",by build the resilience of the poor and those ...,"[[extreme, events], [poor, vulnerable], [reduc..."
...,...,...,...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, ...",16.3,"[enhance, cooperation, access, science, techno...",enhance cooperation on and access to science t...,"[[global, technology], [among, existing], [inn..."
127,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",16.4,"[promote, development, transfer, dissemination...",promote the development transfer dissemination...,"[[development], [promote], [transfer, dissemin..."
128,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,16.5,"[fully, operationalize, technology, bank, scie...",fully operationalize the technology bank and s...,"[[enabling, technology], [technology, bank], [..."
129,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",17.1,"[enhance, capacity, building, support, increas...",by enhance capacity building support to increa...,"[[relevant, national], [availability, high], [..."


In [64]:
# Apply the function to each row of the DataFrame
sdg_sem = sdg_sem.apply(add_elements, axis=1)
sdg_sem

,Goal No.,Goal,Targets,Target_id,new_targets,new_targets_str,keywords
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1,"[eradicate, extreme, poverty, people, everywhe...",by eradicate extreme poverty for all people ev...,"[[poverty, people], [measured, people], [peopl..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2,"[reduce, least, half, men, woman, child, age, ...",by reduce at least by half the proportion of m...,"[[reduce, least], [according, national], [acco..."
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3,"[implement, nationally, appropriate, social, p...",implement nationally appropriate social protec...,"[[appropriate, social], [protection, systems],..."
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4,"[ensure, men, woman, particular, poor, vulnera...",by ensure that all men and women in particular...,"[[well], [particular, poor], [financial, servi..."
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5,"[build, resilience, poor, vulnerable, situatio...",by build the resilience of the poor and those ...,"[[extreme, events], [poor, vulnerable], [reduc..."
...,...,...,...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, ...",16.3,"[enhance, cooperation, access, science, techno...",enhance cooperation on and access to science t...,"[[global, technology], [among, existing], [inn..."
127,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",16.4,"[promote, development, transfer, dissemination...",promote the development transfer dissemination...,"[[development], [promote], [transfer, dissemin..."
128,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,16.5,"[fully, operationalize, technology, bank, scie...",fully operationalize the technology bank and s...,"[[enabling, technology], [technology, bank], [..."
129,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",17.1,"[enhance, capacity, building, support, increas...",by enhance capacity building support to increa...,"[[relevant, national], [availability, high], [..."


In [65]:
sdg_sem['vec'] = sdg_sem['keywords'].apply(sent_vec_avg)
sdg_sem

,Goal No.,Goal,Targets,Target_id,new_targets,new_targets_str,keywords,vec
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1,"[eradicate, extreme, poverty, people, everywhe...",by eradicate extreme poverty for all people ev...,"[[poverty, people], [measured, people], [peopl...","[[-0.2723599970340729, 0.22970100492238998, 0...."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2,"[reduce, least, half, men, woman, child, age, ...",by reduce at least by half the proportion of m...,"[[reduce, least], [according, national], [acco...","[[-0.0706150010228157, 0.3670649975538254, 0.0..."
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3,"[implement, nationally, appropriate, social, p...",implement nationally appropriate social protec...,"[[appropriate, social], [protection, systems],...","[[-0.6093049943447113, -0.26964499801397324, -..."
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4,"[ensure, men, woman, particular, poor, vulnera...",by ensure that all men and women in particular...,"[[well], [particular, poor], [financial, servi...","[[-0.13508999347686768, 0.3590700030326843, 0...."
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5,"[build, resilience, poor, vulnerable, situatio...",by build the resilience of the poor and those ...,"[[extreme, events], [poor, vulnerable], [reduc...","[[0.07290600053966045, 0.157795000821352, 0.27..."
...,...,...,...,...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, ...",16.3,"[enhance, cooperation, access, science, techno...",enhance cooperation on and access to science t...,"[[global, technology], [among, existing], [inn...","[[0.023704998195171356, -0.07121049985289574, ..."
127,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",16.4,"[promote, development, transfer, dissemination...",promote the development transfer dissemination...,"[[development], [promote], [transfer, dissemin...","[[-0.10136999934911728, -0.27173998951911926, ..."
128,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,16.5,"[fully, operationalize, technology, bank, scie...",fully operationalize the technology bank and s...,"[[enabling, technology], [technology, bank], [...","[[0.02382499724626541, -0.09990700334310532, 0..."
129,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",17.1,"[enhance, capacity, building, support, increas...",by enhance capacity building support to increa...,"[[relevant, national], [availability, high], [...","[[-0.09511799737811089, 0.2875400111079216, -0..."


In [66]:
new_df['vec'] = new_df.apply(sent_vec, axis=1)

In [67]:
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,new_description,new_description_str,vec
0,3.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Nada Offices, Nada Offices Village Accountant ...","[nada, office, village, accountant, circle, ho...",nada offices nada offices village accountant c...,"[[-0.07122399657964706, -0.3697200119495392, -..."
1,4.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Va Circles, Nada Offices Village Accountant Ci...","[va, circle, nada, office, village, accountant...",va circles nada offices village accountant cir...,"[[-0.1175599992275238, -0.5297099947929382, 0...."
2,5.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Hoblies, Nada Offices Village Accountant Circl...","[hoblies, nada, office, village, accountant, c...",hoblies nada offices village accountant circle...,"[[-0.07122399657964706, -0.3697200119495392, -..."
3,6.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Grama Panchayaths, Nada Offices Village Accoun...","[grama, panchayaths, nada, office, village, ac...",grama panchayaths nada offices village account...,"[[0.09846500307321548, -0.4200499951839447, -0..."
4,7.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"No.of Taluks, Nada Offices Village Accountant ...","[taluks, nada, office, village, accountant, ci...",noof taluks nada offices village accountant ci...,"[[-0.18463000655174255, -1.0946999788284302, -..."
...,...,...,...,...,...,...,...,...,...
1162,1169.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Total,Number,Total Agrl. Land Holder Total, Ag...","[agrl, land, holder, agricultural, holding, ar...",total number total agrl land holder total agri...,"[[-0.13323000073432922, -0.3314499855041504, -..."
1163,1170.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Male,Area,Total Agrl. Land Holder Total, Agric...","[male, area, agrl, land, holder, agricultural,...",male area total agrl land holder total agricul...,"[[-0.324180006980896, 0.4799000024795532, -0.1..."
1164,1171.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Female,Area,Total Agrl. Land Holder Total, Agr...","[female, area, agrl, land, holder, agricultura...",female area total agrl land holder total agric...,"[[-0.22990000247955322, 0.4291200041770935, -0..."
1165,1172.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Institutions,Area,Total Agrl. Land Holder Tota...","[institution, area, agrl, land, holder, agricu...",institutions area total agrl land holder total...,"[[-0.035638000816106796, -0.29197999835014343,..."


In [68]:
missing_words_set = list(set(missing_words))
print(len(missing_words_set), missing_words_set)

76 [('KAG', 'loantaken'), ('KAG', 'pomagranate'), ('KAG', 'thousandnuts'), ('KAG', 'sesamum'), ('KAG', 'creeches'), ('KAG', 'kutera'), ('KAG', 'ariculated'), ('KAG', 'famalies'), ('KAG', 'tubewell'), ('KAG', 'agrl'), ('KAG', 'agril'), ('KAG', 'suraksha'), ('KAG', 'referal'), ('KAG', 'anganawadi'), ('KAG', 'rsin'), ('KAG', 'ployment'), ('KAG', 'vehical'), ('KAG', 'districtwise'), ('KAG', 'mandays'), ('KAG', 'otal'), ('KAG', 'sectorwise'), ('KAG', 'sqkms'), ('KAG', 'allopathy'), ('KAG', 'estp'), ('KAG', 'drbrambedkar'), ('KAG', 'rainydays'), ('KAG', 'omini'), ('KAG', 'bhagyalakshmi'), ('KAG', 'benificiaries'), ('KAG', 'agricensus'), ('KAG', 'indigenious'), ('KAG', 'lpcd'), ('KAG', 'selfemployment'), ('KAG', 'rainguage'), ('KAG', 'yojane'), ('KAG', 'multiaxled'), ('KAG', 'rikshaws'), ('KAG', 'totalworkers'), ('KAG', 'avare'), ('KAG', 'bengalgram'), ('KAG', 'jananni'), ('KAG', 'hepatities'), ('KAG', 'ipsets'), ('KAG', 'gddp'), ('KAG', 'awnganawadi'), ('KAG', 'mahathma'), ('KAG', 'nddp'), (

In [69]:
miss_added_set = list(set([item for sublist in miss_added for item in sublist]))
print(len(miss_added_set), miss_added_set)

76 ['yojane', 'indigenious', 'bengalgram', 'benificiaries', 'sterlisation', 'anganawadi', 'estp', 'tubewell', 'bhagyajyoti', 'mandays', 'sesamum', 'tubectomy', 'jananni', 'intonnes', 'thousandnuts', 'kutera', 'omini', 'allopathy', 'nrlm', 'guarentee', 'drbrambedkar', 'ashraya', 'avare', 'nigerseed', 'referal', 'rsin', 'mtonnes', 'nitrozen', 'rikshaws', 'delivaries', 'totalworkers', 'hoblies', 'ariculated', 'raingauge', 'sqkms', 'ofemployment', 'lpcd', 'rainguage', 'sapota', 'loantaken', 'kscard', 'proporationof', 'anthyodaya', 'hepatities', 'gddp', 'creeches', 'ployment', 'rainfallfrom', 'famalies', 'districtwise', 'handpump', 'mahathma', 'nddp', 'woked', 'suraksha', 'rainydays', 'facilitiesin', 'vehical', 'awnganawadi', 'ipsets', 'bhagyalakshmi', 'selfemployment', 'mathsasraya', 'agril', 'mgnregs', 'sectorwise', 'pomagranate', 'multiaxled', 'intons', 'distrubuted', 'otal', 'raods', 'agrl', 'shakt', 'agricensus', 'notavailable']


In [70]:
sdg_count_threshold = sdg_sem.copy()

In [71]:
min_val_list, top_n_index_list = find_similarity_semantic(sdg_count_threshold, new_df, 10)

In [72]:
sdg_count_threshold['top_n_index'] = top_n_index_list
sdg_count_threshold['min_values'] = min_val_list
sdg_count_threshold

,Goal No.,Goal,Targets,Target_id,new_targets,new_targets_str,keywords,vec,top_n_index,min_values
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1,"[eradicate, extreme, poverty, people, everywhe...",by eradicate extreme poverty for all people ev...,"[[poverty, people], [measured, people], [peopl...","[[-0.2723599970340729, 0.22970100492238998, 0....","[132, 133, 376, 710, 711, 869, 870, 871, 875, ...","[0.8575334699106769, 0.5624979888636186, 0.629..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2,"[reduce, least, half, men, woman, child, age, ...",by reduce at least by half the proportion of m...,"[[reduce, least], [according, national], [acco...","[[-0.0706150010228157, 0.3670649975538254, 0.0...","[594, 869, 511, 872, 873, 874, 875, 956, 833, ...","[0.6002113658853019, 0.5844719405576027, 0.574..."
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3,"[implement, nationally, appropriate, social, p...",implement nationally appropriate social protec...,"[[appropriate, social], [protection, systems],...","[[-0.6093049943447113, -0.26964499801397324, -...","[198, 659, 660, 661, 662, 663, 664, 665, 666, ...","[0.8510072365400712, 0.7242761885702063, 1.000..."
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4,"[ensure, men, woman, particular, poor, vulnera...",by ensure that all men and women in particular...,"[[well], [particular, poor], [financial, servi...","[[-0.13508999347686768, 0.3590700030326843, 0....","[169, 170, 171, 172, 173, 174, 882, 883, 903, ...","[0.7045711183911056, 0.5746299980541799, 0.678..."
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5,"[build, resilience, poor, vulnerable, situatio...",by build the resilience of the poor and those ...,"[[extreme, events], [poor, vulnerable], [reduc...","[[0.07290600053966045, 0.157795000821352, 0.27...","[963, 964, 965, 966, 967, 968, 969, 970, 971, ...","[0.6873756433137852, 0.527061401761758, 0.5316..."
...,...,...,...,...,...,...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, ...",16.3,"[enhance, cooperation, access, science, techno...",enhance cooperation on and access to science t...,"[[global, technology], [among, existing], [inn...","[[0.023704998195171356, -0.07121049985289574, ...","[198, 659, 660, 661, 662, 663, 664, 665, 666, ...","[0.617751855870593, 0.5874149255443618, 0.5796..."
127,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",16.4,"[promote, development, transfer, dissemination...",promote the development transfer dissemination...,"[[development], [promote], [transfer, dissemin...","[[-0.10136999934911728, -0.27173998951911926, ...","[836, 837, 839, 841, 842, 845, 854, 857, 859, ...","[1.0000000000000009, 0.5658785348581976, 0.488..."
128,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,16.5,"[fully, operationalize, technology, bank, scie...",fully operationalize the technology bank and s...,"[[enabling, technology], [technology, bank], [...","[[0.02382499724626541, -0.09990700334310532, 0...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 28,...","[0.5640035546795967, 0.7908522812373742, 0.583..."
129,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",17.1,"[enhance, capacity, building, support, increas...",by enhance capacity building support to increa...,"[[relevant, national], [availability, high], [...","[[-0.09511799737811089, 0.2875400111079216, -0...","[511, 872, 873, 874, 875, 956, 844, 845, 49, 5...","[0.5559906842694793, 0.8048098255082116, 0.577..."


In [73]:
sdg_min_value = sdg_count_threshold.copy()

In [74]:
sdg_count_threshold_all = finding_attr_details(sdg_count_threshold, new_df)
sdg_count_threshold_all

,Goal No.,Goal,Targets,Target_id,new_targets,new_targets_str,keywords,vec,top_n_index,min_values,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1,"[eradicate, extreme, poverty, people, everywhe...",by eradicate extreme poverty for all people ev...,"[[poverty, people], [measured, people], [peopl...","[[-0.2723599970340729, 0.22970100492238998, 0....","[132, 133, 376, 710, 711, 869, 870, 871, 875, ...","[0.8575334699106769, 0.5624979888636186, 0.629...",54,"[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[3, 3, 5, 11, 11, 14, 14, 14, 14, 3, 1, 1, 1, ...","[3.1, 3.1, 5.4, 11.6, 11.6, 14.3, 14.3, 14.3, ...","[Working,Raingauge Stations, No.of Rainguage R...","[Rainfall, Rainfall, Animal Husbandry, Health ...",[No.of Rainguage Raingauge Stations Annual Nor...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2,"[reduce, least, half, men, woman, child, age, ...",by reduce at least by half the proportion of m...,"[[reduce, least], [according, national], [acco...","[[-0.0706150010228157, 0.3670649975538254, 0.0...","[594, 869, 511, 872, 873, 874, 875, 956, 833, ...","[0.6002113658853019, 0.5844719405576027, 0.574...",60,"[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[10, 14, 9, 14, 14, 14, 14, 17, 13, 13, 13, 13...","[10.11, 14.3, 9.2, 14.3, 14.3, 14.3, 14.3, 17....","[Men,No. of Lecturers, No.of P U Colleges Stud...","[Education, Rural Development & Panchayat Raj,...",[No.of P U Colleges Students and Lecturers Yea...
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3,"[implement, nationally, appropriate, social, p...",implement nationally appropriate social protec...,"[[appropriate, social], [protection, systems],...","[[-0.6093049943447113, -0.26964499801397324, -...","[198, 659, 660, 661, 662, 663, 664, 665, 666, ...","[0.8510072365400712, 0.7242761885702063, 1.000...",55,"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[4, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12...","[4.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11.3...","[Total Pulses including Tur Bengalgram,Pulses ...","[Agriculture, Horticulture and Sericulture, He...","[Area under principal crops Year 2015-16, Allo..."
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4,"[ensure, men, woman, particular, poor, vulnera...",by ensure that all men and women in particular...,"[[well], [particular, poor], [financial, servi...","[[-0.13508999347686768, 0.3590700030326843, 0....","[169, 170, 171, 172, 173, 174, 882, 883, 903, ...","[0.7045711183911056, 0.5746299980541799, 0.678...",33,"[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[4, 4, 4, 4, 4, 4, 14, 14, 15, 13, 13, 13, 13,...","[4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 14.5, 14.5, 15....","[Nos,Wells,2015-16 Net area irrigated Hectares...","[Agriculture, Horticulture and Sericulture, Ag...",[Gross and Net area Irrigated under Different ...
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5,"[build, resilience, poor, vulnerable, situatio...",by build the resilience of the poor and those ...,"[[extreme, events], [poor, vulnerable], [reduc...","[[0.07290600053966045, 0.157795000821352, 0.27...","[963, 964, 965, 966, 967, 968, 969, 970, 971, ...","[0.6873756433137852, 0.527061401761758, 0.5316...",60,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 5, 5,...","[17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17....","[Birth,Registered events Year 2016, Registrati...","[Miscellaneous, Miscellaneous, Miscellaneous, ...","[Registration of Birth and Death, Registration..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, .

In [75]:
sdg_copy = sdg_sem.copy()

In [76]:
sdg_count_threshold_all_goals = process_result_semantic(sdg_count_threshold_all, sdg_copy)

In [77]:
sdg_count_threshold_all.to_csv(absolute_path_semantic, sep=";", index=False)

In [78]:
sdg_count_threshold_all_goals.to_csv(absolute_path_semantic_unroll, sep=";", index=False)

In [79]:
sdg_count_threshold_all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,top_n_goal_id,top_n_target_id
0,140.0,3,Rainfall,3.1,No.of Rainguage Raingauge Stations Annual Norm...,"Working,Raingauge Stations, No.of Rainguage Ra...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
1,141.0,3,Rainfall,3.1,No.of Rainguage Raingauge Stations Annual Norm...,"Not Working,Raingauge Stations, No.of Rainguag...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
2,384.0,5,Animal Husbandry,5.4,Fisheries Year 2016-17,"Capacity Tonnes day,Ice Plants, Fisheries Year...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
3,717.0,11,Health & Family welfare Services,11.6,Different Health Facilities As on 31-3-2017 in...,"Nos.,24 hours and 7 days working Hospitals, Di...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
4,718.0,11,Health & Family welfare Services,11.6,Different Health Facilities As on 31-3-2017 in...,"No.of delivaries in this Hospital,24 hours and...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
...,...,...,...,...,...,...,...,...,...,...
6424,719.0,11,Health & Family welfare Services,11.6,Different Health Facilities As on 31-3-2017 in...,No.of benificiaries in Jananni Suraksha Yojana...,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
6425,720.0,11,Health & Family welfare Services,11.6,Different Health Facilities As on 31-3-2017 in...,"No.of benificiaries medical kit, Different Hea...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
6426,581.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"No.of Govt Primary Schools,Infrastructure Faci...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
6427,582.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Boys Toilet,Infrastructure Facilities in Prima...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2


In [80]:
sdg_count_sim = sdg_sem.copy()

In [81]:
top_index = find_similarity_semantic_threshold(sdg_min_value, new_df, 0.025)

In [82]:
sdg_count_sim['top_n_index'] = top_index

In [83]:
sdg_count_sim_all = finding_attr_details(sdg_count_sim, new_df)
sdg_count_sim_all

,Goal No.,Goal,Targets,Target_id,new_targets,new_targets_str,keywords,vec,top_n_index,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1,"[eradicate, extreme, poverty, people, everywhe...",by eradicate extreme poverty for all people ev...,"[[poverty, people], [measured, people], [peopl...","[[-0.2723599970340729, 0.22970100492238998, 0....","[132, 133, 376, 710, 711, 869, 870, 871, 875, ...",136,"[140.0, 141.0, 384.0, 717.0, 718.0, 876.0, 877...","[3, 3, 5, 11, 11, 14, 14, 14, 14, 3, 3, 3, 1, ...","[3.1, 3.1, 5.4, 11.6, 11.6, 14.3, 14.3, 14.3, ...","[Working,Raingauge Stations, No.of Rainguage R...","[Rainfall, Rainfall, Animal Husbandry, Health ...",[No.of Rainguage Raingauge Stations Annual Nor...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2,"[reduce, least, half, men, woman, child, age, ...",by reduce at least by half the proportion of m...,"[[reduce, least], [according, national], [acco...","[[-0.0706150010228157, 0.3670649975538254, 0.0...","[594, 869, 511, 872, 873, 874, 875, 956, 833, ...",349,"[602.0, 876.0, 519.0, 879.0, 880.0, 881.0, 882...","[10, 14, 9, 14, 14, 14, 14, 17, 13, 13, 13, 13...","[10.11, 14.3, 9.2, 14.3, 14.3, 14.3, 14.3, 17....","[Men,No. of Lecturers, No.of P U Colleges Stud...","[Education, Rural Development & Panchayat Raj,...",[No.of P U Colleges Students and Lecturers Yea...
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3,"[implement, nationally, appropriate, social, p...",implement nationally appropriate social protec...,"[[appropriate, social], [protection, systems],...","[[-0.6093049943447113, -0.26964499801397324, -...","[198, 659, 660, 661, 662, 663, 664, 665, 666, ...",317,"[206.0, 667.0, 668.0, 669.0, 670.0, 670.1, 671...","[4, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 16...","[4.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11.3, 11.3...","[Total Pulses including Tur Bengalgram,Pulses ...","[Agriculture, Horticulture and Sericulture, He...","[Area under principal crops Year 2015-16, Allo..."
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4,"[ensure, men, woman, particular, poor, vulnera...",by ensure that all men and women in particular...,"[[well], [particular, poor], [financial, servi...","[[-0.13508999347686768, 0.3590700030326843, 0....","[169, 170, 171, 172, 173, 174, 882, 883, 903, ...",491,"[177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 889...","[4, 4, 4, 4, 4, 4, 14, 14, 15, 13, 13, 13, 13,...","[4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 14.5, 14.5, 15....","[Nos,Wells,2015-16 Net area irrigated Hectares...","[Agriculture, Horticulture and Sericulture, Ag...",[Gross and Net area Irrigated under Different ...
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5,"[build, resilience, poor, vulnerable, situatio...",by build the resilience of the poor and those ...,"[[extreme, events], [poor, vulnerable], [reduc...","[[0.07290600053966045, 0.157795000821352, 0.27...","[963, 964, 965, 966, 967, 968, 969, 970, 971, ...",208,"[970.0, 971.0, 972.0, 973.0, 974.0, 975.0, 976...","[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 1...","[17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17....","[Birth,Registered events Year 2016, Registrati...","[Miscellaneous, Miscellaneous, Miscellaneous, ...","[Registration of Birth and Death, Registration..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, ...",16.3,"[enhance, cooperation, access, science, techno...",enhance cooperation on and access to science t...,"[[global, technology], [among, existing], [inn...","[[0.023704998195171356, -0.07121049985289574, ...","[198, 659, 660, 661, 662, 663, 664, 665, 666, ...",417

In [84]:
sdg_copy2 = sdg_sem.copy()

In [85]:
sdg_count_sim_all_goals = process_result_semantic(sdg_count_sim_all, sdg_copy2)

In [86]:
sdg_count_sim_all.to_csv(absolute_path_sem_threshold, sep=";", index=False)

In [87]:
sdg_count_sim_all_goals.to_csv(absolute_path_sem_threshold_unroll, sep=";", index=False)

In [88]:
sdg_count_sim_all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,top_n_goal_id,top_n_target_id
0,140.0,3,Rainfall,3.1,No.of Rainguage Raingauge Stations Annual Norm...,"Working,Raingauge Stations, No.of Rainguage Ra...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
1,141.0,3,Rainfall,3.1,No.of Rainguage Raingauge Stations Annual Norm...,"Not Working,Raingauge Stations, No.of Rainguag...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
2,384.0,5,Animal Husbandry,5.4,Fisheries Year 2016-17,"Capacity Tonnes day,Ice Plants, Fisheries Year...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
3,717.0,11,Health & Family welfare Services,11.6,Different Health Facilities As on 31-3-2017 in...,"Nos.,24 hours and 7 days working Hospitals, Di...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
4,718.0,11,Health & Family welfare Services,11.6,Different Health Facilities As on 31-3-2017 in...,"No.of delivaries in this Hospital,24 hours and...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
...,...,...,...,...,...,...,...,...,...,...
38026,586.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Ramp,Infrastructure Facilitiesin Primary Schoo...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
38027,587.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Compound,Infrastructure Facilitiesin Primary S...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
38028,588.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Drinking Water,Infrastructure Facilitiesin Pri...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
38029,589.0,10,Education,10.9,Infrastructure Facilities in Primary schools Y...,"Library,Infrastructure Facilitiesin Primary Sc...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
